In [1]:
import pandas as pd
import pywikibot
from bs4 import BeautifulSoup, SoupStrainer
import requests
import re
from Constants import *
import pickle

### 1.Extract potential QIDS for a newspaper 

In [29]:
def get_QIDS(newspaper):
    url = f'https://www.wikidata.org/w/index.php?search={newspaper} \
    &title=Special%3ASearch&fulltext=1&ns0=1&ns120=1'
    page = requests.get(url)    
    data = page.text
    soup = BeautifulSoup(data)
    QIDS = []
    for link in soup.find_all('a'):
        link = link.get('href')
        if(link is not None and link.startswith("/wiki/Q")):
            QID =  link.replace('/wiki/','')
            QIDS.append(QID)
        
    return QIDS

In [36]:
df = pd.read_csv(FILE_NEWSPAPER_SPEAKER_COUNT,compression='bz2')

newspapers = df['newspaper'].unique()


array(['1011now', '101greatgoals', '1045theteam', ..., 'zerotackle',
       'zigwheels', 'zip06'], dtype=object)

In [37]:
print(len(newspapers))
newspapers

7362


array(['1011now', '101greatgoals', '1045theteam', ..., 'zerotackle',
       'zigwheels', 'zip06'], dtype=object)

In [38]:
#Dictionnary of newspaper and found QIDS
newspapers_QIDS = {}

for i,newspaper in enumerate(newspapers):
    if(i%10==0):
        print(f'Iteration {i+1}')
    QIDS = get_QIDS(newspaper)
    print(newspaper,QIDS)
    newspapers_QIDS[newspaper]=QIDS

Iteration 1
1011now ['Q6334998', 'Q2078885', 'Q80948465', 'Q62624260', 'Q42788041', 'Q101566522']
101greatgoals []
1045theteam ['Q7956286']
1070thefan ['Q7949418']
107jamz ['Q6331512']
10news ['Q3191396', 'Q16931724', 'Q31197620', 'Q6823090', 'Q6671082', 'Q1973', 'Q161053', 'Q355', 'Q37156', 'Q1995', 'Q2014', 'Q12439', 'Q1537', 'Q23548', 'Q47272', 'Q8458', 'Q1008', 'Q251', 'Q193755']
1130thetiger ['Q6340648']
1380kcim ['Q6326840']
13abc ['Q2386816', 'Q80056', 'Q781365', 'Q2283', 'Q159683', 'Q438406', 'Q187846', 'Q1401', 'Q700722', 'Q181003', 'Q165257', 'Q304330', 'Q939838', 'Q7315', 'Q186969', 'Q46602', 'Q415167', 'Q361208', 'Q89238011', 'Q9438']
1428elm ['Q88077135', 'Q223596', 'Q21481859', 'Q21558717', 'Q56881478', 'Q56650644', 'Q30299897', 'Q56743449', 'Q56741441', 'Q56744903', 'Q27350225', 'Q56744297', 'Q56744835', 'Q56743750', 'Q30277959', 'Q56740902', 'Q56740201', 'Q56743786', 'Q56743062', 'Q27335792']
Iteration 11
14850 ['Q85189953', 'Q83901193', 'Q67650301', 'Q63002074', 'Q1861

aafp ['Q4742888', 'Q30263994', 'Q72848114', 'Q74107866', 'Q4743731', 'Q89902631', 'Q80838910', 'Q95549895', 'Q73022807', 'Q94381786', 'Q84014996', 'Q74369100', 'Q71096882', 'Q67542674', 'Q89902797', 'Q72154507', 'Q81587923', 'Q97526505', 'Q82798973', 'Q69801033']
aaj ['Q286152', 'Q18109318', 'Q23925252', 'Q1962825', 'Q1722580', 'Q3396358', 'Q1496227', 'Q4120101', 'Q4661278', 'Q22682062', 'Q19869542', 'Q4661300', 'Q4661305', 'Q65122880', 'Q4661284', 'Q48724048', 'Q4661285', 'Q22079618', 'Q21503791', 'Q64768455']
aappublications ['Q7159238', 'Q96589992', 'Q24091343', 'Q66444726', 'Q23922790', 'Q23922897', 'Q23922784', 'Q23922949']
aarp ['Q286252', 'Q463410', 'Q59978745', 'Q58537543', 'Q59874654', 'Q59976486', 'Q56711340', 'Q19043619', 'Q59910158', 'Q59964182', 'Q24033155', 'Q59956960', 'Q59981632', 'Q4649931', 'Q59735219', 'Q58516881', 'Q59956887', 'Q20245735', 'Q106609797', 'Q54862753']
abacusnews []
abajournal ['Q102182527', 'Q106196620', 'Q4650020']
abbynews ['Q4664307']
abc ['Q169889

advanced-television ['Q4686311', 'Q4686316', 'Q3774494', 'Q637801', 'Q81195050', 'Q99849980', 'Q99878472', 'Q99878390', 'Q99849521', 'Q99879049', 'Q3255817', 'Q2544194', 'Q41590428', 'Q67343980', 'Q85782424', 'Q3008253', 'Q278375', 'Q278185', 'Q87455756']
advancedsciencenews []
advantagenews ['Q28231550']
adventistreview ['Q2825387', 'Q4686449']
adventuresinpoortaste []
advertiser ['Q4686731', 'Q100276955', 'Q100276945', 'Q100276954', 'Q100276942', 'Q100276944', 'Q100276943', 'Q100276952', 'Q100276953', 'Q100276956', 'Q18679989', 'Q1519246', 'Q10355417', 'Q941351', 'Q56027834', 'Q6905642', 'Q100297805', 'Q6912784', 'Q7712710', 'Q89920271']
advocate ['Q380075', 'Q15253558', 'Q4686881', 'Q7712724', 'Q27717171', 'Q100140556', 'Q100276963', 'Q100276967', 'Q100276969', 'Q100276973', 'Q100276970', 'Q96734946', 'Q100276964', 'Q12606028', 'Q100276966', 'Q64965491', 'Q100276974', 'Q100276972', 'Q100276965']
advocate-online ['Q15759232', 'Q752361', 'Q29698483', 'Q29698638', 'Q29698639', 'Q296985

alaskahighwaynews ['Q4708553']
albawaba ['Q4703568']
albernivalleynews []
albionnewsonline []
aldergrovestar ['Q4713664']
Iteration 191
alderleyedge ['Q104389501', 'Q54987463']
aldianews []
aledotimesrecord ['Q56236493']
aleteia ['Q16010657', 'Q47170042', 'Q95717717', 'Q28791067', 'Q2840730', 'Q59296693', 'Q7722783', 'Q88531736', 'Q92755348', 'Q81489226', 'Q3545810', 'Q37668490', 'Q57059552', 'Q93071696', 'Q99373372', 'Q48542327', 'Q1317994', 'Q48535539', 'Q91992463', 'Q51803384']
alextimes ['Q56027627', 'Q51486546']
algemeiner ['Q1961754', 'Q38787426']
alibi ['Q468947', 'Q1618533', 'Q409493', 'Q9603694', 'Q4273984', 'Q4725653', 'Q4725652', 'Q4725654', 'Q4725664', 'Q682418', 'Q18786487', 'Q4725657', 'Q16020951', 'Q16960959', 'Q34644473', 'Q4489331', 'Q3874816', 'Q11850865', 'Q16746301', 'Q3985627']
aliran ['Q650383', 'Q65084132', 'Q96718060', 'Q222516', 'Q47521', 'Q143089', 'Q4726861', 'Q166620', 'Q650035', 'Q110701', 'Q695793', 'Q1387659', 'Q816321', 'Q18957', 'Q108777669', 'Q373342',

areadevelopment []
areanews []
areawidenews []
arizona ['Q503419', 'Q816', 'Q58684', 'Q49221', 'Q670897', 'Q58691', 'Q58711', 'Q58696', 'Q58686', 'Q58771', 'Q58694', 'Q58688', 'Q58692', 'Q16556', 'Q58774', 'Q51685', 'Q58683', 'Q58712', 'Q58698', 'Q18575']
Iteration 311
arizonasports ['Q6338484', 'Q6335629']
arizonawildcats ['Q4791461', 'Q4791464', 'Q3174614', 'Q23302579', 'Q4791460', 'Q4791465', 'Q18148132']
arkansas-catholic ['Q25230330', 'Q10835482', 'Q99216010', 'Q105322302', 'Q25001725', 'Q25207615', 'Q5053180', 'Q8676420', 'Q7588947', 'Q65759634', 'Q7589275', 'Q7589268', 'Q7589750', 'Q7587833', 'Q7590111', 'Q94988360', 'Q94988942', 'Q85791417', 'Q7591600', 'Q25084888']
arkansasbusiness ['Q4791751', 'Q55936089']
arkansasfight []
arkansasonline ['Q94311480', 'Q4791763']
arkansasrazorbacks ['Q4791842', 'Q3098181', 'Q4791844', 'Q3866524', 'Q4791820', 'Q16961952', 'Q43079505', 'Q48742678', 'Q108893122', 'Q63322950']
arktimes ['Q4791888']
arlingtontimes []
arlnow []
Iteration 321
armeni

...

blu-ray ['Q47770', 'Q20072147', 'Q885525', 'Q100946497', 'Q19953724', 'Q15456942', 'Q795736', 'Q795747', 'Q11232614', 'Q493883', 'Q16787644', 'Q55523988', 'Q19829025', 'Q17230702', 'Q19867291', 'Q4928648', 'Q21829638', 'Q19867307', 'Q18391668', 'Q7906636']
blueandgreentomorrow []
bluebirdbanter []
bluegrasstoday []
bluemountainsgazette ['Q24185423']
Iteration 711
blueshirtbanter []
bmmagazine ['Q5001757']
bnn ['Q56505', 'Q809590', 'Q407820', 'Q2664252', 'Q1431005', 'Q1017576', 'Q2060884', 'Q56456871', 'Q1872521', 'Q30315082', 'Q17362588', 'Q99601134', 'Q66923658', 'Q106104847', 'Q58339093', 'Q58698081', 'Q30315072', 'Q106106812', 'Q15872832', 'Q79266640']
bnnbloomberg ['Q1017576', 'Q109513622']
boatingindustry []
bobsguide []
bobvila []
bocamag []
bocanewsnow []
bocaratontribune ['Q56030671']
Iteration 721
bodyandsoul []
bodyshopbusiness []
boereport []
bognor ['Q890621', 'Q890615', 'Q14157870', 'Q61038923', 'Q2450822', 'Q89577525', 'Q4937672', 'Q3137665', 'Q24674460', 'Q2278205', 'Q38

bsminfo []
bsu ['Q805122', 'Q424332', 'Q4887885', 'Q891082', 'Q30280314', 'Q2788303', 'Q816377', 'Q104904742', 'Q1101040', 'Q2418908', 'Q3578258', 'Q3446076', 'Q4484261', 'Q4855385', 'Q24780017', 'Q24780045', 'Q24740027', 'Q24777346', 'Q24783614', 'Q1003463']
bt ['Q917', 'Q593786', 'Q7108', 'Q361431', 'Q218871', 'Q819558', 'Q327821', 'Q133862', 'Q3941683', 'Q40001', 'Q751803', 'Q27823178', 'Q4836753', 'Q4836748', 'Q29305', 'Q27823194', 'Q27823193', 'Q54798420', 'Q27823191', 'Q27823201']
btlnews []
bu ['Q49110', 'Q226025', 'Q4097588', 'Q3519221', 'Q20800254', 'Q3913321', 'Q4997520', 'Q39747949', 'Q626918', 'Q11058213', 'Q3534400', 'Q317846', 'Q997297', 'Q34772497', 'Q25203912', 'Q4891776', 'Q22602608', 'Q69513653', 'Q4915566', 'Q1484018']
buccaneers ['Q320476', 'Q7720462', 'Q30765837', 'Q5329506', 'Q5084177', 'Q16847897', 'Q8829639', 'Q946448', 'Q8829600', 'Q1644156', 'Q1939123', 'Q7854852', 'Q100320314', 'Q2389721', 'Q2927421', 'Q5109363', 'Q106918736', 'Q1034381', 'Q7073974', 'Q279713

cambridge-news ['Q100288085', 'Q5025487', 'Q85750225', 'Q103831031', 'Q55667466', 'Q51498983', 'Q96718926', 'Q68976812', 'Q58110617', 'Q58777872', 'Q58775526', 'Q58729846', 'Q5382707', 'Q6029185', 'Q41965974', 'Q4035428', 'Q95887291', 'Q17658969']
cambridgeday []
cambridgeindependent []
cambstimes []
camdenadvertiser []
camdenchat []
camdencourier []
camdennewjournal ['Q5025795']
camera ['Q15328', 'Q313600', 'Q62927', 'Q1208175', 'Q196353', 'Q20888659', 'Q196342', 'Q20741022', 'Q313614', 'Q29576', 'Q161833', 'Q335095', 'Q1723004', 'Q738160', 'Q192234', 'Q20683640', 'Q313742', 'Q209918']
campaignasia []
Iteration 961
campaignindia []
campaignlive ['Q5027734', 'Q100288102']
campaignseries []
campbellrivermirror ['Q5028137']
campdenfb []
campusinsiders ['Q25349798']
campussafetymagazine ['Q6861148']
campustimes []
canada ['Q16', 'Q1121436', 'Q138345', 'Q488523', 'Q2569593', 'Q176', 'Q1904', 'Q383590', 'Q1989', 'Q36312', 'Q1068063', 'Q2009', 'Q14624136', 'Q208635', 'Q94301', 'Q196615', 'Q1

celebsnow []
celebuzz ['Q5057789', 'Q554091']
celiac ['Q11088', 'Q5058032', 'Q54165657', 'Q519368', 'Q34109907', 'Q68048746', 'Q33932672', 'Q1995284', 'Q34410846', 'Q28243044', 'Q28281691', 'Q22306535', 'Q5058033', 'Q42606472', 'Q34566764', 'Q33457749', 'Q897760', 'Q35004094', 'Q28185658', 'Q33964135']
celticsblog []
cemnet []
centralbanking ['Q66344', 'Q2007090', 'Q79449886', 'Q8901', 'Q944085', 'Q53536', 'Q806651', 'Q283785', 'Q806703', 'Q190223', 'Q333101', 'Q806714', 'Q244411', 'Q74687', 'Q190230', 'Q564652', 'Q249203', 'Q685918']
centralchronicle ['Q24947033']
centraljersey ['Q56567335', 'Q7244987']
centralmaine ['Q95882087', 'Q6389544', 'Q6912862']
Iteration 1091
centraltelegraph []
centralwesterndaily ['Q5062031']
centreforaviation []
cessnockadvertiser []
cfcnet []
cfl ['Q1032175', 'Q351387', 'Q525359', 'Q802704', 'Q207571', 'Q802703', 'Q4582888', 'Q2342646', 'Q674250', 'Q171408', 'Q3238986', 'Q16762885', 'Q16986671', 'Q11707124', 'Q3062569', 'Q2619257', 'Q856218', 'Q66384942',

ConnectionError: HTTPSConnectionPool(host='www.wikidata.org', port=443): Max retries exceeded with url: /w/index.php?search=chad%20%20%20%20%20&title=Special%3ASearch&fulltext=1&ns0=1&ns120=1 (Caused by NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x7f96598ff1c0>: Failed to establish a new connection: [Errno 60] Operation timed out'))

In [56]:
new_start_index = len(newspapers_QIDS)
newspapers_new_start = newspapers[new_start_index:]
for i,newspaper in enumerate(newspapers_new_start):
    if(i%10==0):
        print(f'Iteration {i+new_start_index+1}')
    QIDS = get_QIDS(newspaper)
    print(newspaper,QIDS)
    newspapers_QIDS[newspaper]=QIDS

Iteration 4259
newarkadvertiser ['Q7016603']
newatlas ['Q7672596', 'Q106643136']
newbostonpost []
newburytoday ['Q7017256']
newcastlestar ['Q7017485']
newclimate ['Q77077066']
newdelhitimes []
newfoodmagazine []
newgamenetwork ['Q7007734']
newhamrecorder ['Q7018146']
Iteration 4269
newhavenindependent ['Q28404112', 'Q7912113', 'Q86826270']
newindianexpress ['Q1955025', 'Q104175581', 'Q106903888', 'Q96179268']
newkerala ['Q6961677']
newmarketjournal []
newmatilda ['Q7018604']
neworleanscitybusiness ['Q7010707']
neworleanssaints ['Q172435', 'Q16973696', 'Q22023429', 'Q18702427', 'Q4626332', 'Q48843404', 'Q2657408', 'Q60525883', 'Q104869610']
newportbeachindy []
newrepublic ['Q1329873', 'Q104649112', 'Q104764912', 'Q60786968', 'Q3454432', 'Q76570207', 'Q47515142', 'Q63126919', 'Q932554']
newrytimes []
Iteration 4279
news ['Q38926', 'Q848629', 'Q28549308', 'Q374160', 'Q335017', 'Q192283', 'Q48340', 'Q186068', 'Q1160945', 'Q1193236', 'Q270389', 'Q12020', 'Q1962634', 'Q220509', 'Q11032', 'Q4

newyorktheatreguide []
newyorkupstate []
newzimbabwe []
nextbigfuture []
nextcity ['Q7020959', 'Q55457512', 'Q39476364']
nextgov ['Q69060002']
nextgreencar []
nextshark []
Iteration 4369
nexusnewspaper []
nfca ['Q107132779', 'Q107121565', 'Q106959122', 'Q107133154', 'Q7644959', 'Q22341370', 'Q52951', 'Q16650148', 'Q47497664', 'Q1430959', 'Q14405706', 'Q27972895', 'Q61906153', 'Q705427', 'Q3438970']
nfl ['Q1215884', 'Q32096', 'Q443821', 'Q60526042', 'Q47090681', 'Q74108831', 'Q6953707', 'Q2430783', 'Q16266784', 'Q1395918', 'Q28127991', 'Q21993818', 'Q724353', 'Q2190688', 'Q68294787', 'Q18698858', 'Q1883234']
nflmocks []
nflspinzone []
ngnews ['Q7753906']
nh1 ['Q16460688', 'Q54930506', 'Q15175938', 'Q19624721', 'Q54669932', 'Q24777254', 'Q66843665', 'Q35814907', 'Q61600227', 'Q642922', 'Q37137447', 'Q39227296', 'Q44995419', 'Q35185644', 'Q7949427', 'Q7956414', 'Q7957746', 'Q7952366', 'Q61600238', 'Q7958320']
nhbr ['Q91437340', 'Q17071457', 'Q72498307', 'Q31323999', 'Q28232210', 'Q7307320

northweststar []
northwichguardian ['Q42892274']
Iteration 4459
norwalkreflector ['Q22256455']
nothinbutnets []
nottinghampost ['Q7063679', 'Q7063691']
nouse ['Q7063931', 'Q4822386', 'Q65241407', 'Q81438792', 'Q72331468', 'Q48414120', 'Q71469486', 'Q76463448', 'Q73722501', 'Q90159306', 'Q40339469', 'Q190991', 'Q375364', 'Q53387943', 'Q64415731']
novanewsnow ['Q7738773']
novinite ['Q7064915', 'Q139886']
nowboxing []
nowthisnews ['Q16927746', 'Q27988243']
nprillinois ['Q7956600']
nraila []
Iteration 4469
nrdc ['Q6955118', 'Q24382670', 'Q841895', 'Q18030217', 'Q30272344', 'Q20631618', 'Q71889740', 'Q6955119', 'Q62291509', 'Q85787555', 'Q71889750', 'Q71895478', 'Q62326588', 'Q59031033', 'Q69711268', 'Q17012175', 'Q73677002', 'Q58958275', 'Q24811098', 'Q71889024']
nreionline []
nrl ['Q1140630', 'Q407310', 'Q18030220', 'Q24606630', 'Q1499258', 'Q24408111', 'Q18252721', 'Q108532478', 'Q1334435', 'Q64091192', 'Q29739167', 'Q28595099', 'Q6961668', 'Q27722589', 'Q91919115', 'Q91420564', 'Q168619

officialcharts ['Q66811907', 'Q193259', 'Q719415', 'Q18067871', 'Q7079576', 'Q107370988']
offshore-technology ['Q86972083', 'Q96328879', 'Q352989', 'Q57511410', 'Q56364706', 'Q689880', 'Q56367351', 'Q14954495', 'Q596976', 'Q96720041', 'Q101552148', 'Q4686263', 'Q67105327', 'Q7079691', 'Q59665376', 'Q59170124', 'Q56030271', 'Q15763742', 'Q98825794', 'Q97618722']
offsiderulepodcast []
offtackleempire []
offtheball []
ohscanada []
ohsonline ['Q29008833', 'Q26343480', 'Q26343491', 'Q26343432', 'Q26343450', 'Q26343446', 'Q26343460', 'Q26343499', 'Q26215635', 'Q26343427', 'Q26343486', 'Q26343423', 'Q26343456', 'Q26343412', 'Q66672600', 'Q26343438', 'Q24847075', 'Q26343484', 'Q26343466', 'Q26343476']
oilandgasinvestor []
oilandgastechnology []
Iteration 4559
oilprice []
oilvoice []
ok ['Q1649', 'Q2205649', 'Q218704', 'Q217844', 'Q2017022', 'Q1896106', 'Q1123836', 'Q736107', 'Q12501114', 'Q7072619', 'Q37511456', 'Q30080289', 'Q108370360', 'Q25326634', 'Q1666000', 'Q202996', 'Q108370394', 'Q946

pamplinmedia ['Q7129439', 'Q8032450', 'Q6477242', 'Q16916433', 'Q22313035']
panampost ['Q22907590']
panda ['Q33602', 'Q41960', 'Q1786478', 'Q37325049', 'Q23936885', 'Q129193', 'Q3790388', 'Q7536283', 'Q3361970', 'Q24849776', 'Q7130428', 'Q4832749', 'Q11886595', 'Q36654309', 'Q17452428', 'Q17563096', 'Q5100585', 'Q2187393', 'Q25290603', 'Q21819113']
pandct []
panthers ['Q330120', 'Q204623', 'Q478436', 'Q7199342', 'Q7199341', 'Q7131824', 'Q7131827', 'Q5425988', 'Q93696975', 'Q5547756', 'Q5330213', 'Q7238024', 'Q1474044', 'Q3110096', 'Q1781094', 'Q3374818', 'Q16202119', 'Q24892346', 'Q2645029', 'Q7058443']
papermag ['Q7132584']
parade ['Q657449', 'Q7134083', 'Q7134092', 'Q1189399', 'Q255959', 'Q51404', 'Q2051330', 'Q3894997', 'Q1970030', 'Q2051292', 'Q4048500', 'Q3895002', 'Q43414587', 'Q95612785', 'Q964089', 'Q7134089', 'Q3894999', 'Q95572553', 'Q95612711', 'Q15880480']
paradisepost ['Q7134342']
parentherald []
Iteration 4689
parkbugle ['Q56795957']
parkcitiespeople []
parkers ['Q9660231

...

wcsh6 ['Q24483']
wctrib ['Q7984692']
wctv ['Q2536991', 'Q7949609', 'Q16186798', 'Q25619124', 'Q4072748', 'Q11062368', 'Q7590654']
Iteration 6919
wcyy ['Q7948238', 'Q34684', 'Q4206729', 'Q15445294']
wdam ['Q7948267', 'Q7948266', 'Q30295145']
wdbj7 ['Q7948284', 'Q94616677']
wdef ['Q7948355', 'Q7948357', 'Q7948356']
wdet ['Q7948371', 'Q101364694', 'Q101359374', 'Q101362203', 'Q101365051', 'Q101366448', 'Q75778766']
wdio ['Q7948412', 'Q88213846']
wdiy ['Q7948414', 'Q101364998', 'Q101365021', 'Q101365031', 'Q101365013', 'Q101365025']
wdtv ['Q7948603', 'Q7956815', 'Q100272881', 'Q1881437', 'Q17230562', 'Q33859949', 'Q39091385', 'Q59006792', 'Q48790837', 'Q29539383', 'Q2919674', 'Q15365500']
we-heart ['Q10393281', 'Q106688218', 'Q5691767', 'Q10379', 'Q54993704', 'Q54993703', 'Q7768758', 'Q7977585', 'Q7773874', 'Q6547165', 'Q37099340', 'Q72207288', 'Q27890237', 'Q37362029', 'Q1117936', 'Q37004826', 'Q38875783', 'Q37744271', 'Q36005398', 'Q93576006']
wealthdaily []
Iteration 6929
wealthmanageme

wheels24 ['Q50845469', 'Q95576898', 'Q93202426', 'Q37418026', 'Q5706835', 'Q91885995', 'Q734000', 'Q60311564', 'Q80653543', 'Q97366955', 'Q88020769', 'Q59812510', 'Q49164531', 'Q55722137', 'Q46935731', 'Q57785261', 'Q46969558', 'Q84056935', 'Q81373415', 'Q50528349']
which-50 ['Q27860958', 'Q169960', 'Q14819271', 'Q60583251', 'Q474090', 'Q6496962', 'Q181754', 'Q102254106', 'Q69455153', 'Q11395', 'Q1752891', 'Q2456754', 'Q26824554', 'Q24339631', 'Q927148', 'Q7603694', 'Q482715', 'Q24314363', 'Q2720759', 'Q28259492']
whidbeynewstimes ['Q7993571']
whig ['Q108700', 'Q42183', 'Q397092', 'Q100287225', 'Q2164376', 'Q109475906', 'Q106671613', 'Q2660437', 'Q14339337', 'Q100287959', 'Q5056353', 'Q96279805', 'Q928635', 'Q100287152', 'Q100287201', 'Q100297052', 'Q100284984', 'Q15043276', 'Q100301764', 'Q100301201']
whio ['Q7950202', 'Q7950205', 'Q14716849', 'Q7950204', 'Q34887563', 'Q106251303', 'Q83362419', 'Q33717230', 'Q420499', 'Q104666469', 'Q57483722', 'Q96095998', 'Q40502203', 'Q33465006']
w

wkyt ['Q7952054', 'Q18395875', 'Q1133545', 'Q24933389', 'Q211852', 'Q26256744', 'Q18772049', 'Q18844221', 'Q21176707']
wkzo ['Q7952076', 'Q7952079']
wlfi ['Q7952204', 'Q7949360', 'Q77887723']
wlni ['Q7952367']
wlox ['Q7952407', 'Q14693441']
wlrn ['Q7952452', 'Q7952454', 'Q101358501', 'Q7952453', 'Q101360409', 'Q107352666', 'Q20087932', 'Q1057085', 'Q18394006', 'Q15430827', 'Q9370466', 'Q85834952', 'Q20992327']
wltribune []
wltz ['Q7952518', 'Q14686909', 'Q7335865', 'Q317844', 'Q3378602', 'Q62795270', 'Q106949107', 'Q445195', 'Q39035612', 'Q30384319']
wm ['Q298999', 'Q13406463', 'Q9369373', 'Q504423', 'Q19794482', 'Q2552219', 'Q344838', 'Q26884324', 'Q30134734', 'Q8038430', 'Q64093121', 'Q3487403', 'Q4167836', 'Q54994215', 'Q922344', 'Q12344159', 'Q59549568', 'Q1736285', 'Q1337474', 'Q63285045']
Iteration 7129
wmagazine ['Q11339926', 'Q1853094', 'Q50327016', 'Q103405436', 'Q106360255']
wmbfnews ['Q7952661']
wmcactionnews5 ['Q7952679', 'Q5228425']
wmdt ['Q7952728', 'Q7952730', 'Q103759',

wruf ['Q7955338', 'Q7955341', 'Q7955339', 'Q7955340', 'Q7085382', 'Q59254306']
Iteration 7219
wrvo ['Q7955392', 'Q101386019', 'Q101394071', 'Q101355656', 'Q101361727', 'Q7955391', 'Q7955381', 'Q7955387', 'Q7957898', 'Q80087507']
wsau ['Q7955477', 'Q7955479', 'Q7955478', 'Q18151179']
wsaw ['Q7955483', 'Q6929540', 'Q44511789', 'Q59218083', 'Q51916775']
wsaz ['Q7955485', 'Q7955484', 'Q33848955']
wsbradio ['Q7955527', 'Q7955500']
wsbtv ['Q7955494', 'Q27067898']
wscountytimes []
wsfa ['Q7955589', 'Q7955591', 'Q7955593']
wshu ['Q7955657', 'Q7955659', 'Q7955658', 'Q101357521', 'Q101360093', 'Q101362245', 'Q101362353', 'Q101362412', 'Q7954763', 'Q7955973', 'Q7957958', 'Q19915214']
wsiltv ['Q7955673', 'Q106643136']
Iteration 7229
wsj ['Q164746', 'Q395469', 'Q4407375', 'Q220015', 'Q7955688', 'Q99689947', 'Q20880937', 'Q65027925', 'Q105045556', 'Q20328156', 'Q91134763', 'Q64153924', 'Q78511305', 'Q59861330', 'Q933390', 'Q76575510', 'Q1138934', 'Q17654383', 'Q10961498']
wsjm ['Q7955693', 'Q7955697

wzzm13 ['Q7958441']
x17online ['Q52', 'Q939416', 'Q39379', 'Q43297', 'Q483959', 'Q17989936', 'Q8142', 'Q9589', 'Q16186120', 'Q958', 'Q217305', 'Q669166', 'Q1075339', 'Q15913890', 'Q47596', 'Q383541', 'Q16829376', 'Q157020', 'Q189507', 'Q564426']
xcmag []
xconomy ['Q8043395', 'Q6944906', 'Q95218956']
xda-developers ['Q262900', 'Q68484894', 'Q105853554', 'Q105854898']
xinhuanet ['Q11081577', 'Q204839', 'Q59816890', 'Q108660842', 'Q11081573', 'Q71339', 'Q10473364', 'Q55602467', 'Q4727816', 'Q92157950', 'Q6487717', 'Q19843231', 'Q108846948']
xxlmag ['Q55416426', 'Q2063886']
yachtingworld ['Q8046448']
yakimaherald ['Q8047089']
Iteration 7309
yale ['Q49112', 'Q1143289', 'Q2846167', 'Q1568434', 'Q255147', 'Q1928992', 'Q2295845', 'Q175606', 'Q2599077', 'Q1961140', 'Q2556397', 'Q3240101', 'Q2992915', 'Q499138', 'Q8047353', 'Q8047423', 'Q951340', 'Q6352575', 'Q1138025']
yaleclimateconnections []
yaledailynews ['Q2599070', 'Q107167553', 'Q107331518', 'Q107331531', 'Q81426384']
yalibnan ['Q8046238

In [60]:
newspapers_QIDS_temp = newspapers_QIDS
PICKLE_NEWSPAPER_QID = 'processed_data/newspaper_qids.pickle'

In [108]:
# with open(PICKLE_NEWSPAPER_QID_FINAL, 'wb') as handle:
#     pickle.dump(newspapers_QIDS, handle, protocol=pickle.HIGHEST_PROTOCOL)

### 2.From potential QIDS, get correct QID as well as owner name if exists

In [70]:
owned_by_key = 'P127'
official_website_key = 'P856'
site = pywikibot.Site('en', 'wikipedia')
repo = site.data_repository()

def extract_website_owner(newspaper,QIDS):
    current_QID = None
    owner = None
    official_website = None
    
    for QID in QIDS:
        current_QID = QID
        item = pywikibot.ItemPage(repo, QID)
        if(item.exists()):
            if(item.isRedirectPage()):
                item = page.getRedirectTarget()
                
            claims = item.get()['claims'].toJSON()# Provides dict["property_key"->property]
            
            official_website_property = claims.get(official_website_key)
            if(official_website_property!=None):
                official_website = official_website_property[0]['mainsnak']['datavalue']['value']
                
                if(newspaper in official_website):
                    owned_by_property = claims.get(owned_by_key)#Return None if property not found
                    if(owned_by_property!=None):
                        owner_id = owned_by_property[0]['mainsnak']['datavalue']['value']['numeric-id']
                        #Use ItemPage to obtain a dictionnary
                        data_found = pywikibot.ItemPage(repo, "Q" + str(owner_id))
                        #Get title in english
                        owner = data_found.get()['labels']['en']
                    break
                    
    return current_QID, official_website, owner

In [65]:
newspapers_QIDS = None
with open(PICKLE_NEWSPAPER_QID, 'rb') as handle:
    newspapers_QIDS = pickle.load(handle)

print(len(newspapers_QIDS))

7362


In [71]:
newspapers_info = {}

for i, (newspaper,QIDS) in enumerate(newspapers_QIDS.items()):
    if(i%10==0):
        print(f'Iteration {i+1}')
    infos= extract_website_owner(newspaper,QIDS)
    newspapers_info[newspaper] = infos
    print(newspaper,infos)

Iteration 1
1011now ('Q6334998', 'https://www.1011now.com/', None)
101greatgoals (None, None, None)
1045theteam ('Q7956286', 'http://www.1045theteam.com', 'Townsquare Media')
1070thefan ('Q7949418', 'http://www.1070thefan.com/', 'Emmis Communications')
107jamz ('Q6331512', 'http://www.107jamz.com', 'Townsquare Media')
10news ('Q3191396', 'http://www.10news.com/', 'E. W. Scripps Company')
1130thetiger ('Q6340648', 'http://www.kwkhonline.com', None)
1380kcim ('Q6326840', 'http://www.1380kcim.com/', None)
13abc ('Q2386816', 'http://www.13abc.com/', 'Gray Television')
1428elm ('Q27335792', 'http://www.x-menfirstclassmovie.com', None)
Iteration 11
14850 ('Q23533712', None, None)
14news ('Q7949314', 'http://www.14news.com/', None)
15rounds ('Q46017852', None, None)
1888pressrelease (None, None, None)
220triathlon (None, None, None)
225batonrouge (None, None, None)
247sports ('Q16973497', 'https://247sports.com/', 'ViacomCBS Streaming')
247wallst ('Q68896999', 'https://247wallst.com/', None)


afro ('Q63959138', None, None)
aft ('Q30264800', 'http://www.aft-online.net/home/die-akademie/', None)
afterdawn ('Q11849854', 'http://afterdawn.com', None)
aftermarketnews (None, None, None)
Iteration 151
agassizharrisonobserver (None, None, None)
agmetalminer (None, None, None)
agri-pulse ('Q81558016', None, None)
agriculture ('Q50815382', 'https://www.mdpi.com/journal/agriculture', None)
agriland (None, None, None)
agrinews-pubs (None, None, None)
agweek (None, None, None)
agwired (None, None, None)
ahmedabadmirror (None, None, None)
ahvalnews (None, None, None)
Iteration 161
ai-cio ('Q4696465', 'http://www.ai-cio.com', None)
aijac ('Q4651929', None, None)
aikenstandard ('Q4696943', None, None)
aim ('Q2003140', 'http://www.myspace.com/aimflight602', None)
ainonline ('Q48797965', 'https://www.ainonline.com', None)
aip ('Q4651980', 'http://proceedings.aip.org/', None)
airalamo (None, None, None)
airdrieecho (None, None, None)
airforcetimes ('Q4697970', 'https://www.airforcetimes.com/'

archpaper ('Q7714192', 'http://www.archpaper.com', None)
arcticicehockey (None, None, None)
arcurrent (None, None, None)
ardrossanherald (None, None, None)
areadevelopment (None, None, None)
areanews (None, None, None)
areawidenews (None, None, None)
arizona ('Q503419', 'https://www.arizona.edu', None)
Iteration 311
arizonasports ('Q6338484', 'http://arizonasports.com/', 'Bonneville International')
arizonawildcats ('Q4791461', 'https://arizonawildcats.com/sports/football', None)
arkansas-catholic ('Q25230330', 'http://www.arkansas-catholic.org/', None)
arkansasbusiness ('Q55936089', None, None)
arkansasfight (None, None, None)
arkansasonline ('Q4791763', 'https://www.arkansasonline.com/', 'WEHCO Media')
arkansasrazorbacks ('Q4791842', 'https://arkansasrazorbacks.com/sport/m-footbl/', None)
arktimes ('Q4791888', 'http://arktimes.com/', None)
arlingtontimes (None, None, None)
arlnow (None, None, None)
Iteration 321
armenianweekly ('Q4793242', 'http://armenianweekly.com/', None)
armidalee

badgerherald ('Q7715317', 'http://badgerherald.com/', None)
badlefthook (None, None, None)
badzine (None, None, None)
bahai ('Q22679', 'http://www.bahai.org/', None)
bainbridgereview ('Q55671356', None, None)
bakingbusiness (None, None, None)
balkaninsight ('Q4851082', 'http://www.balkaninsight.com/', None)
Iteration 471
balldurham (None, None, None)
ballerstatus (None, None, None)
ballinaadvocate (None, None, None)
ballparkdigest ('Q5056857', 'http://www.kernels.com', None)
balls ('Q260464', 'http://www.edballs.co.uk/', None)
ballstatesports ('Q4851219', 'http://ballstatesports.com', None)
balonnebeacon ('Q23640813', None, None)
baltic-course ('Q40088924', 'http://www.baltic-course.com/', None)
baltimorebeatdown (None, None, None)
baltimorepostexaminer ('Q55667227', None, None)
Iteration 481
baltimoreravens ('Q276539', 'http://www.baltimoreravens.com', 'Eric DeCosta')
bamsmackpow (None, None, None)
banana1015 ('Q7957122', 'http://banana1015.com', 'Townsquare Media')
banbridgeleader (N

binghamton ('Q213814', 'http://www.cityofbinghamton.com/', None)
bio-itworld (None, None, None)
biodieselmagazine (None, None, None)
biofuelsdigest (None, None, None)
biography ('Q28025983', 'https://www.biography.com/', None)
biologicaldiversity ('Q3196675', 'http://biologicaldiversity.org/', None)
biomassmagazine (None, None, None)
Iteration 641
biomedcentral ('Q463494', 'https://www.biomedcentral.com/', None)
biopharma-reporter (None, None, None)
biopharmadive (None, None, None)
biospectrumasia (None, None, None)
birdswatcher (None, None, None)
birminghammail ('Q4916749', 'https://www.birminghammail.co.uk/', None)
birminghamtimes ('Q101504021', 'http://www.birminghamtimes.com', None)
bit ('Q4835742', 'http://www.bit-teatergarasjen.no/', None)
bit-tech ('Q4918667', 'http://www.bit-tech.net', None)
bitcoin ('Q131723', 'https://bitcoincore.org/en/download', None)
Iteration 651
bitcoinmagazine ('Q24590022', 'http://www.bitcoinmagazine.com/', None)
biv ('Q2894589', 'http://www.bellbivdev

...

juneauempire ('Q6312479', 'http://juneauempire.com/', None)
juneesoutherncross (None, None, None)
juniorminingnetwork (None, None, None)
junkee ('Q19874827', 'http://junkeemedia.com/', None)
jurist ('Q6109136', 'http://jurist.org/', None)
Iteration 3191
just-auto ('Q10541532', 'http://www.academyautomuseum.org/', None)
just-drinks ('Q33889630', None, None)
just-food ('Q82200940', 'https://www.justeatplc.com/', None)
just-style ('Q92939503', 'https://martirenti.com', None)
justarsenal (None, None, None)
justblogbaby (None, None, None)
justia ('Q39048191', 'https://www.justia.com/', None)
justjared (None, None, None)
justjaredjr (None, None, None)
justmeans (None, None, None)
Iteration 3201
justsecurity (None, None, None)
jweekly ('Q6103861', 'http://www.jweekly.com/', None)
k2radio ('Q6339315', 'http://www.k2radio.com/', 'Townsquare Media')
kaaltv ('Q3191309', 'http://www.kaaltv.com/', None)
kadn ('Q6324824', 'https://kadn.com/', 'Entertainment Studios')
kait8 ('Q3191320', 'http://www.k

komu ('Q4230308', 'http://www.komuvnyz.com', None)
koptalk (None, None, None)
Iteration 3341
koreaherald ('Q562824', 'http://www.koreaherald.com/', None)
kosu ('Q6335249', 'http://www.kosu.org', 'Oklahoma State University–Stillwater')
kotaku ('Q1083295', 'https://kotaku.com', 'G/O Media')
kotaradio ('Q6335263', 'http://www.kotaradio.com/', None)
kowb1290 ('Q6335323', 'http://www.kowb1290.com/', 'Townsquare Media')
kpax ('Q6335429', 'http://www.kpax.com/', None)
kpbs ('Q6335464', 'https://www.kpbs.org/', None)
kpcnews ('Q6335499', 'http://kpcnews.com', None)
kpcw ('Q6335496', 'http://www.kpcw.org', None)
kpel965 ('Q6335538', 'http://www.kpel965.com', 'Townsquare Media')
Iteration 3351
kplctv ('Q6335637', 'http://www.kplctv.com/', None)
kpopstarz (None, None, None)
kpug1170 ('Q6335958', 'http://www.kpug1170.com/', None)
kqed ('Q766792', 'http://www.kqed.org/tv/', None)
krakowpost ('Q6435948', 'http://www.krakowpost.com/', None)
krbd ('Q14680063', 'http://www.krbd.org', None)
krcc ('Q6336

laurinburgexchange ('Q6501830', 'http://www.laurinburgexchange.com', None)
lavendermagazine ('Q6502505', 'http://www.lavendermagazine.com/', None)
law360 ('Q6502903', 'https://www.law360.com/', 'LexisNexis')
Iteration 3501
lawandcrime ('Q104007059', 'https://lawandcrime.com/', 'Abrams Media')
lawcareers ('Q6502904', None, None)
laweekly ('Q952554', 'http://www.laweekly.com', 'Voice Media Group')
lawfareblog ('Q29097894', 'http://www.lawfareblog.com', None)
lawfuel (None, None, None)
lawinsport (None, None, None)
lawndalenews (None, None, None)
lawstreetmedia ('Q39046749', 'https://lawstreetmedia.com/', None)
lawtimesnews (None, None, None)
lawyersweekly (None, None, None)
Iteration 3511
lbc ('Q6457278', 'http://www.lbc.co.uk/', 'Global')
lbknews (None, None, None)
lbl ('Q1133630', 'https://www.lbl.gov/', None)
lbmjournal (None, None, None)
lbpost (None, None, None)
lcctorch (None, None, None)
lcnewschronicle (None, None, None)
leaderlive ('Q7746499', None, None)
leaderpost ('Q3423475',

lovefm ('Q11443195', 'http://lovefm.co.jp/pc/ln_top.php', None)
loverugbyleague ('Q24763743', 'http://www.loverugbyleague.com', None)
lovindublin (None, None, None)
lowellsun ('Q2077538', 'http://lowellsun.com/', 'MediaNews Group')
lowestoftjournal (None, None, None)
lowyat ('Q18210656', 'http://www.lowyat.net/', None)
lowyinterpreter (None, None, None)
loyolamaroon (None, None, None)
Iteration 3671
lpga ('Q27650', 'http://www.lpga.com', None)
lrb ('Q1869064', 'http://www.lrb.co.uk/', None)
lsbf ('Q1869074', 'http://www.lsbf.org.uk', None)
lse ('Q174570', 'http://www.lse.ac.uk', None)
lsusports ('Q1325166', 'https://lsusports.net/', None)
lubavitch ('Q737685', 'https://www.lubavitch.com/', None)
lucire ('Q1873506', 'http://lucire.com/', None)
lucknowsentinel (None, None, None)
lufkindailynews ('Q7749170', 'http://www.lufkindailynews.com/', 'Southern Newspapers')
lutontoday (None, None, None)
Iteration 3681
lutterworthmail (None, None, None)
luxurydaily (None, None, None)
lynnnews (None

KeyError: 'en'

In [73]:
print(len(newspapers_info))

3719


In [103]:
new_start_index = len(newspapers_info)+1 #Bug at one index
newspapers_QIDS_intermediate = list(newspapers_QIDS.items())
newspapers_QIDS_intermediate = newspapers_QIDS_intermediate[new_start_index:]

In [104]:
for i, (newspaper,QIDS) in enumerate(newspapers_QIDS_intermediate):
    if(i%10==0):
        print(f'Iteration {i+new_start_index+1}')
    infos= extract_website_owner(newspaper,QIDS)
    newspapers_info[newspaper] = infos
    print(newspaper,infos)

Iteration 7024
whio ('Q7950205', 'http://whio.com', 'Cox Media Group')
whiskycast (None, None, None)
whistleblowersblog (None, None, None)
whitbygazette ('Q7994167', 'http://www.whitbygazette.co.uk/', 'Johnston Press')
whitecenterblog (None, None, None)
whitecenternow (None, None, None)
whitecourtstar ('Q7995812', 'http://www.whitecourtstar.com/', 'Postmedia Network')
whitehouse ('Q974133', 'http://www.whitehousetx.org/', None)
whitehousedossier (None, None, None)
whitsundaytimes ('Q7996814', None, None)
Iteration 7034
whittierdailynews ('Q7996870', 'https://www.whittierdailynews.com/', 'Digital First Media')
whiznews ('Q7950216', 'https://www.whiznews.com/', None)
whkp ('Q7950239', 'http://www.whkp.com/', None)
whmi ('Q1298480', 'http://whmi.com', None)
who ('Q7817', 'https://www.who.int', None)


who2 ('Q185351', None, None)
whodatdish (None, None, None)
wholefoodsmagazine ('Q106368834', 'https://drinkh2rose.com', None)
wholehogsports (None, None, None)
whowhatwhy ('Q7381095', None, None)
Iteration 7044
whsv ('Q55279', 'http://www.whsv.com/', 'Gray Television')
whtc ('Q7950437', 'http://www.whtc.com/', None)
whtimes ('Q22337566', 'http://www.whtimes.co.uk', None)
whufc ('Q18747', 'http://www.whufc.com/', 'David Sullivan')
whyallanewsonline (None, None, None)
whyy ('Q55457531', 'https://whyy.org', None)
wibc ('Q7950564', 'http://www.wibc.com/', 'Emmis Communications')
wibw ('Q2537278', 'http://www.wibwnewsnow.com/am580/', 'Alpha Media')
wibwnewsnow ('Q2537278', 'http://www.wibwnewsnow.com/am580/', 'Alpha Media')
wibx950 ('Q7950583', 'http://www.wibx950.com/', 'Townsquare Media')
Iteration 7054
wicklownews (None, None, None)
wicnews (None, None, None)
wicz ('Q7950612', 'http://www.wicz.com/', None)
wideopencountry (None, None, None)
wifr ('Q7950652', 'http://www.wifr.com/', None)

wosu ('Q7953991', 'http://wosu.org/radio/', 'Ohio State University')
woub ('Q7954013', 'http://www.woub.org/radio/', 'Ohio University')
wowt ('Q7954042', 'http://www.wowt.com/', 'Gray Television')
wpr ('Q7950033', 'http://www.wpr.org', 'Wisconsin Educational Communications Board')
wpsu ('Q7954484', 'https://radio.wpsu.org/', 'Pennsylvania State University')
wptv ('Q7954513', 'http://www.wptv.com/', None)
wpxi ('Q10985787', 'http://www.wpxi.com/', None)
wqcs ('Q7954628', 'http://www.wqcs.org', 'Indian River State College')
wqow ('Q7954749', 'http://www.wqow.com/', None)
Iteration 7204
wral ('Q7954860', 'http://www.wralfm.com/', 'Capitol Broadcasting Company')
wralsportsfan ('Q7947906', 'http://www.wralsportsfan.com', 'Capitol Broadcasting Company')
wraltechwire (None, None, None)
wranglernews ('Q55665437', None, None)
wrbiradio ('Q7954889', 'http://www.wrbiradio.com', None)
wrcbtv ('Q14710276', 'http://www.wrcbtv.com/', None)
wrdw ('Q7954961', 'http://www.wrdw.com/', None)
wrestlinginc 

ypct ('Q14933746', 'http://trapeziste-creative.com/', None)
ysusports ('Q8058667', 'http://www.ysusports.com/', None)
yubanet (None, None, None)
yukon-news ('Q8060835', 'http://www.yukon-news.com', None)
yumasun ('Q8061046', 'http://www.yumasun.com/', 'RISN Operations')
zacks ('Q20501835', 'https://www.warnerbros.com/movies/justice-league/', None)
zalebs (None, None, None)
Iteration 7354
zap2it ('Q8066356', 'http://www.zap2it.com/', 'Tribune Media')
zatznotfunny (None, None, None)
zawya ('Q105823347', 'https://www.legatum.com/', None)
zdnet ('Q2457578', 'http://zdnet.com/', 'Red Ventures')
zeibiz (None, None, None)
zeldadungeon ('Q2632541', None, None)
zerotackle (None, None, None)
zigwheels ('Q19899301', 'http://www.zigwheels.com/', None)
zip06 ('Q27929737', 'https://www.7-zip.org/', None)


In [109]:
print(len(newspapers_info))

7358


In [115]:
missing = set(newspapers_QIDS.keys()).difference(set(newspapers_info.keys()))

In [116]:
for m in missing:
    newspapers_info[m] = (None,None,None)

In [118]:
NEWSPAPER_INFO_PICKLE ='processed_data/newspaper_infos.pickle'
with open(NEWSPAPER_INFO_PICKLE, 'wb') as handle:
    pickle.dump(newspapers_info, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [123]:
flattened = [[k,v1,v2,v3]for k,(v1,v2,v3) in newspapers_info.items()]

In [131]:
df = pd.DataFrame(flattened,columns = ['newspaper','QID','website','owner'])

In [129]:
df

,newspaper,QID,website,owner
0,1011now,Q6334998,https://www.1011now.com/,None
1,101greatgoals,None,None,None
2,1045theteam,Q7956286,http://www.1045theteam.com,Townsquare Media
3,1070thefan,Q7949418,http://www.1070thefan.com/,Emmis Communications
4,107jamz,Q6331512,http://www.107jamz.com,Townsquare Media
...,...,...,...,...
7357,zip06,Q27929737,https://www.7-zip.org/,None
7358,networkworld,None,None,None
7359,rnews,None,None,None
7360,mamamia,None,None,None


In [135]:
df.to_csv('processed_data/newspaper-owner.csv',index=False)

In [2]:
df = pd.read_csv('processed_data/newspaper-owner.csv')
df

,newspaper,QID,website,owner
0,1011now,Q6334998,https://www.1011now.com/,NaN
1,101greatgoals,NaN,NaN,NaN
2,1045theteam,Q7956286,http://www.1045theteam.com,Townsquare Media
3,1070thefan,Q7949418,http://www.1070thefan.com/,Emmis Communications
4,107jamz,Q6331512,http://www.107jamz.com,Townsquare Media
...,...,...,...,...
7357,zip06,Q27929737,https://www.7-zip.org/,NaN
7358,networkworld,NaN,NaN,NaN
7359,rnews,NaN,NaN,NaN
7360,mamamia,NaN,NaN,NaN


In [4]:
group_size = df[~df['owner'].isna()].groupby('owner').size()

In [6]:
group_size[group_size>1].sum()

625